## Aqui vamos realizar o estudo de como funciona o  Pytorch


Esse código é referente ao guia de uso do pytorch disponivel em: https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html


Primeiramente vamos realizar os devidos imports das bibliotecas.

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

Agora iremos realiza o download de dois dos nossos datasets.
 Um será de dados para o treinamento e o outro para de teste

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


 Definimos nossos dataloaders com um tamanho de Batch de 64.

In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


Agora verificamos se utilizaremos a CPU ou GPU para o treinamento.


In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cpu device


Definimos uma classe chamada NeuralNetwork onde ela herda de __nn.Module__,nós definimos as camadas de conexão na função ____init____ e definimos como os dados serão transmitidos através da rede na função __forward__.

In [ ]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Para treinar o modelo, é necessario uma __Função de perda__ e a utilização do pacote __torch.optim__

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

Nossa função __train__ irá realizar previsões e aplicar a __retropropagação__.

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

Agora verificamos a perfomance do nosso modelo contra o dataset de testes para certificamos que está havendo aprendizagem.

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Agora interamos e observamos as predições do nosso modelo atráves dos nossos loops (__epochs__).

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.301728  [    0/60000]
loss: 2.290676  [ 6400/60000]
loss: 2.268679  [12800/60000]
loss: 2.263580  [19200/60000]
loss: 2.257564  [25600/60000]
loss: 2.213328  [32000/60000]
loss: 2.228776  [38400/60000]
loss: 2.185699  [44800/60000]
loss: 2.189754  [51200/60000]
loss: 2.157249  [57600/60000]
Test Error: 
 Accuracy: 42.1%, Avg loss: 2.149804 

Epoch 2
-------------------------------
loss: 2.156303  [    0/60000]
loss: 2.149370  [ 6400/60000]
loss: 2.089719  [12800/60000]
loss: 2.106976  [19200/60000]
loss: 2.060158  [25600/60000]
loss: 1.982441  [32000/60000]
loss: 2.022568  [38400/60000]
loss: 1.927657  [44800/60000]
loss: 1.949358  [51200/60000]
loss: 1.877439  [57600/60000]
Test Error: 
 Accuracy: 52.0%, Avg loss: 1.869760 

Epoch 3
-------------------------------
loss: 1.904833  [    0/60000]
loss: 1.873433  [ 6400/60000]
loss: 1.760838  [12800/60000]
loss: 1.800927  [19200/60000]
loss: 1.695609  [25600/60000]
loss: 1.636733  [32000/600

Salvando o modelo


In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


Carregando o modelo.

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
